<a href="https://colab.research.google.com/github/SpandanaDC/ZETA1_SIH25143/blob/main/footpressure_sensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install micromlgen


  Preparing metadata (setup.py) ... done
  Created wheel for micromlgen: filename=micromlgen-1.1.28-py3-none-any.whl size=32152 sha256=2344697db7e0b246fafdaadb442947581268f5ca640fbc88dfab29bad02a619d
  Stored in directory: /root/.cache/pip/wheels/16/02/8a/3a8b533174e4f7691a8fd72dab4493fb6819b79f8fcc1d18a6
Successfully built micromlgen


In [2]:
# ==========================================
# CELL 1: LIBRARIES & CONFIGURATION
# ==========================================
import os
import glob
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from micromlgen import port

class Config:
    # Folder where you uploaded the .txt files
    DATA_DIR = "/content/gait_data"
    MODEL_SAVE_PATH = "orthosis_model.pkl"

    # Sensor indices within ONE FOOT (8 sensors, 0-7)
    IDX_HEEL    = 0          # Sensor 1 (heel)
    IDX_LATERAL = [1, 3, 5]  # outer edge
    IDX_MEDIAL  = [2, 4, 6]  # inner edge
    IDX_TOE     = 7          # toe

    # Augmentation Multipliers
    SUPINATION_LATERAL_BOOST = 1.8
    SUPINATION_MEDIAL_REDUCE = 0.2
    PRONATION_MEDIAL_BOOST   = 1.8
    PRONATION_LATERAL_REDUCE = 0.2

print("✅ Libraries imported and Config set.")


✅ Libraries imported and Config set.


In [3]:
# ==========================================
# CELL 2: LOAD RAW FILES (BOTH FEET) + NORMALIZE
# ==========================================
def load_raw_files():
    """
    Reads all .txt files in DATA_DIR.
    Each file has 19 columns:
      time, L1..L8, R1..R8, Lsum, Rsum
    We create TWO samples per row: one for left, one for right.
    Final shape: [N_samples, 8]
    """
    pattern = os.path.join(Config.DATA_DIR, "*.txt")
    files = sorted(glob.glob(pattern))

    if not files:
        print(f"[ERROR] No .txt files found in {Config.DATA_DIR}")
        return np.empty((0, 8))

    all_steps = []

    for path in files:
        df = pd.read_csv(path, sep=r"\s+", header=None)
        n_cols = df.shape[1]
        print(f"{os.path.basename(path)} → {n_cols} columns")

        if n_cols == 19:
            # time | L1..L8 | R1..R8 | Lsum | Rsum
            left  = df.iloc[:, 1:9].values   # L1..L8
            right = df.iloc[:, 9:17].values  # R1..R8

            # Each row gives two foot-samples
            all_steps.append(left)
            all_steps.append(right)
        elif n_cols == 8:
            # Already single-foot format
            all_steps.append(df.values)
        else:
            print(f"[WARN] Skipping {path}, unsupported cols: {n_cols}")

    if not all_steps:
        print("[ERROR] No valid samples after parsing.")
        return np.empty((0, 8))

    data = np.vstack(all_steps)
    print("📌 Loaded single-foot samples:", data.shape)
    return data


def normalize_data(data):
    """
    Column-wise min-max normalization to [0, 1].
    This removes scale differences, but model
    will mostly use ratios anyway.
    """
    data = data.astype(float)
    min_val = data.min(axis=0, keepdims=True)
    max_val = data.max(axis=0, keepdims=True)
    denom = (max_val - min_val)
    denom[denom == 0] = 1.0
    norm = (data - min_val) / denom
    return norm

print("✅ Loader & normalizer ready.")


✅ Loader & normalizer ready.


In [4]:
# ==========================================
# CELL 3: FEATURE ENGINEERING + MID-STANCE FILTER
# ==========================================
def extract_features(step8):
    """
    step8: np.array of shape (8,)
    Returns a feature vector that depends ONLY on
    PRESSURE DISTRIBUTION (ratios), not on body weight.
    """
    total = step8.sum()
    if total <= 1e-6:
        # no contact, return zeros (will mostly be filtered out)
        return np.zeros(7, dtype=float)

    norm = step8 / total  # normalize by total → weight-invariant

    heel_ratio   = norm[Config.IDX_HEEL]
    toe_ratio    = norm[Config.IDX_TOE]
    medial_ratio = norm[Config.IDX_MEDIAL].sum()
    lateral_ratio= norm[Config.IDX_LATERAL].sum()

    # Imbalance indicators (still ratios)
    ml_diff   = medial_ratio - lateral_ratio    # inside - outside
    ht_diff   = heel_ratio - toe_ratio          # back - front
    spread    = norm.std()                      # how uneven is the distribution?

    # We DO NOT include 'total' here → independent of absolute force/weight
    return np.array([
        heel_ratio,
        toe_ratio,
        medial_ratio,
        lateral_ratio,
        ml_diff,
        ht_diff,
        spread
    ], dtype=float)


def filter_midstance(data,
                     heel_low=0.10, heel_high=0.40,
                     toe_low=0.10, toe_high=0.40):
    """
    Keep only samples where both heel and toe have moderate load:
    -> approx. mid-stance.
    """
    kept = []
    for s in data:
        total = s.sum()
        if total <= 1e-6:
            continue

        heel = s[Config.IDX_HEEL]
        toe  = s[Config.IDX_TOE]
        heel_ratio = heel / total
        toe_ratio  = toe  / total

        if (heel_low < heel_ratio < heel_high) and (toe_low < toe_ratio < toe_high):
            kept.append(s)

    kept = np.array(kept)
    print("✅ Mid-stance filtered samples:", kept.shape)
    return kept

print("✅ Feature extraction + mid-stance filter ready.")


✅ Feature extraction + mid-stance filter ready.


In [5]:
# ==========================================
# CELL 4: DATA AUGMENTATION → 3 CLASSES
# ==========================================
def create_abnormal_dataset(midstance_data):
    """
    From each mid-stance step (8 sensors), create:
      - class 0: Stable      (original)
      - class 1: Supination  (outer side boosted, inner reduced)
      - class 2: Pronation   (inner side boosted, outer reduced)
    Returns:
      X: [N_samples, n_features]
      y: [N_samples]
    """
    X = []
    y = []

    print(f"[Augmentation] Using {len(midstance_data)} mid-stance steps")

    for step in midstance_data:
        # --- class 0: stable ---
        X.append(extract_features(step))
        y.append(0)

        # --- supination: roll outward (lateral high, medial low) ---
        sup = step.copy()
        sup[Config.IDX_LATERAL] *= Config.SUPINATION_LATERAL_BOOST
        sup[Config.IDX_MEDIAL]  *= Config.SUPINATION_MEDIAL_REDUCE
        X.append(extract_features(sup))
        y.append(1)

        # --- pronation: roll inward (medial high, lateral low) ---
        pro = step.copy()
        pro[Config.IDX_MEDIAL]  *= Config.PRONATION_MEDIAL_BOOST
        pro[Config.IDX_LATERAL] *= Config.PRONATION_LATERAL_REDUCE
        X.append(extract_features(pro))
        y.append(2)

    X = np.array(X)
    y = np.array(y)
    print("✅ Augmented dataset:", X.shape, " | Label counts:", np.bincount(y))
    return X, y

print("✅ Augmentation with engineered features ready.")


✅ Augmentation with engineered features ready.


In [6]:
# ==========================================
# CELL 5: TRAIN MODEL + EXPORT FOR ESP32
# ==========================================
def train_and_export(X, y):
    print(f"[Training] {len(X)} samples | feature dim = {X.shape[1]}")

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    clf = RandomForestClassifier(
        n_estimators=30,
        max_depth=7,
        min_samples_split=4,
        random_state=42
    )

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"\n✅ Accuracy: {acc*100:.2f}%")
    print("\nClassification Report:")
    print(classification_report(
        y_test, y_pred,
        target_names=["Stable", "Supination", "Pronation"]
    ))

    # Save sklearn model
    joblib.dump(clf, Config.MODEL_SAVE_PATH)
    print(f"[Saved] PC model → {Config.MODEL_SAVE_PATH}")

    # Export to C++ for ESP32 using micromlgen
    c_code = port(clf, classmap={
        0: "STABLE",
        1: "SUPINATE",
        2: "PRONATE"
    })

    with open("model.h", "w") as f:
        f.write(c_code)

    print("💾 Exported ESP32 model → model.h")
    return clf

print("✅ Training + export function ready.")


✅ Training + export function ready.


In [7]:
# ==========================================
# CELL 6: RUN FULL PIPELINE
# ==========================================
print("=== SPINOBAR ML PIPELINE – BOTH FEET, WEIGHT-INVARIANT ===")

raw = load_raw_files()
if raw.shape[0] == 0:
    print("❌ ERROR: No raw samples loaded. Check DATA_DIR and file format.")
else:
    norm = normalize_data(raw)
    print("🔹 Normalized samples:", norm.shape)

    mid = filter_midstance(norm)
    if mid.shape[0] == 0:
        print("❌ ERROR: No mid-stance samples found. Try relaxing thresholds.")
    else:
        X, y = create_abnormal_dataset(mid)
        model = train_and_export(X, y)

        print("\n🎉 DONE: Download model.h and put it into your ESP32 project /include/ folder.")


=== SPINOBAR ML PIPELINE – BOTH FEET, WEIGHT-INVARIANT ===
GaCo01_01.txt → 19 columns
📌 Loaded single-foot samples: (24238, 8)
🔹 Normalized samples: (24238, 8)
✅ Mid-stance filtered samples: (666, 8)
[Augmentation] Using 666 mid-stance steps
✅ Augmented dataset: (1998, 7)  | Label counts: [666 666 666]
[Training] 1998 samples | feature dim = 7

✅ Accuracy: 98.25%

Classification Report:
              precision    recall  f1-score   support

      Stable       0.97      0.98      0.97       133
  Supination       0.98      1.00      0.99       133
   Pronation       1.00      0.97      0.98       134

    accuracy                           0.98       400
   macro avg       0.98      0.98      0.98       400
weighted avg       0.98      0.98      0.98       400

[Saved] PC model → orthosis_model.pkl
💾 Exported ESP32 model → model.h

🎉 DONE: Download model.h and put it into your ESP32 project /include/ folder.
